<a href="https://colab.research.google.com/github/vivekkishore/machinelearning/blob/main/llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LLama index

###Query GPT with LLama index

In [ ]:
# !pip install llama-index
# !pip install llama-index-llms-azure-openai
# !pip install python-dotenv

In [ ]:
llm = AzureOpenAI(
    model=azure_gpt_model_name,
    deployment_name=gpt_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)
response = llm.complete("The sky is blue")
print(response)

Yes, that is correct. The sky appears blue during the day due to the scattering of sunlight by the Earth's atmosphere.


In [ ]:
from llama_index.core.llms import ChatMessage

messages=[
  ChatMessage(role='system',content='You are funny and helpful bot'
    ),
    ChatMessage(role='user',content='why is sky blue?')
]

response=llm.chat(messages)
print(response)

assistant: Well, the sky is blue because it's trying to be cool and stand out from all the other colors. It's like the sky's way of saying, "Hey, look at me, I'm the coolest color around!" Plus, blue is just a really calming and peaceful color, so it's the perfect choice for the sky.


###Using LLama index to query list of JSON

In [ ]:
# !pip install llama-index
# !pip install llama-index-llms-azure-openai
# !pip install llama-index-embeddings-azure-openai
# !pip install sqlite-utils
# !pip install python-dotenv

In [ ]:
data=[
    {'name':'charls','age':10,'city':'Newyork'},
    {'name':'bob','age':15,'city':'delhi'},
    {'name':'alice','age':20,'city':'mumbai'},
     {'name':'john','age':25,'city':'chicago'},
     {'name':'maddie','age':21,'city':'pune'}
     ]

In [ ]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from dotenv import dotenv_values
config = dotenv_values(".env")


azure_openai_key = config["API_KEY"]
azure_openai_endpoint = config["ENDPOINT"]
azure_gpt_model_name=config["GPT_Model_Name"]
gpt_deployment_name=config["Deployment_Name"]
api_version=config['Model_Version']
azure_embed_model_name=config["Embed_Model_Name"]
embedding_deployment_name=config["Embed_Deployment"]

llm = AzureOpenAI(
    model=azure_gpt_model_name,
    deployment_name=gpt_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

embed_model = AzureOpenAIEmbedding(
    model=azure_embed_model_name,
    deployment_name=embedding_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

In [ ]:
from llama_index.core.query_engine import JSONalyzeQueryEngine
#JSON data is being converted to SQL first
engine=JSONalyzeQueryEngine(
    list_of_dict=data,
    llm=llm,
    verbose=True,
)

In [ ]:
result=engine.query("Do anyone live in city bangalore?")
result

Query: Do anyone live in city bangalore?
SQL Query: SELECT COUNT(*) FROM items WHERE city = 'bangalore'
Table Schema: {'name': <class 'str'>, 'age': <class 'int'>, 'city': <class 'str'>}
SQL Response: [{'COUNT(*)': 0}]
Response: No, no one lives in the city of Bangalore.

Response(response='No, no one lives in the city of Bangalore.', source_nodes=[], metadata={'sql_query': "SELECT COUNT(*) FROM items WHERE city = 'bangalore'", 'table_schema': "{'name': <class 'str'>, 'age': <class 'int'>, 'city': <class 'str'>}"})

In [ ]:
print(result)

No, no one lives in the city of Bangalore.


###Query text file with LLamma index



In [ ]:
!pip install llama-index docx2txt pypdf python-dotenv llama-index-embeddings-azure-openai llama-index-llms-azure-openai

In [7]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from dotenv import dotenv_values

config=dotenv_values("val.env")

azure_openai_key = config["API_KEY"]
azure_openai_endpoint = config["ENDPOINT"]
azure_gpt_model_name=config["GPT_Model_Name"]
gpt_deployment_name=config["Deployment_Name"]
api_version=config['Model_Version']
azure_embed_model_name=config["Embed_Model_Name"]
embedding_deployment_name=config["Embed_Deployment"]

llm = AzureOpenAI(
    model=azure_gpt_model_name,
    deployment_name=gpt_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

embed_model = AzureOpenAIEmbedding(
    model=azure_embed_model_name,
    deployment_name=embedding_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

In [8]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [23]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex,StorageContext

documents = SimpleDirectoryReader( input_files=["sample1.txt"]).load_data()
index = VectorStoreIndex.from_documents(documents)

In [15]:
query="What is SP initiated integration?"
query_engine=index.as_query_engine(llm=llm )
answer=query_engine.query(query)

# print(answer.get_formatted_sources())
print("query was:", query)
print("answer is:", answer)

query was: What is SP initiated integration?
answer is: SP initiated integration refers to a type of integration where the service provider (SP) takes the initiative to establish a connection or interaction with another system or service. In this type of integration, the SP is responsible for initiating the communication and sending the necessary data or requests to the other system. This can be done through various protocols and mechanisms, such as API calls, webhooks, or direct messaging. SP initiated integration allows the SP to control the flow of information and initiate actions based on specific triggers or events.


In [ ]:
from llama_index.core import PromptTemplate
template=(
    "I have provided the context below \n"
    "-------------------------\n"
    "You are a funny bot who always starts the answer with one joke \n"
    "---------------------------------- \n"
    "Given this context, please answer this question: {query_str} \n"
)

query="What is SP initiated integration?"
qa_prompt=PromptTemplate(template)

prompt=qa_prompt.format(query_str=query)
# print(prompt)

answer=query_engine.query(prompt)
print(answer)


Why did the scarecrow win an award? Because he was outstanding in his field! 

SP-initiated integration is a login process in which a user navigates to the vendor application, which then redirects the user to Azure AD for authentication. After authentication, the user is redirected back to the application with an authorization token.


#####Using PDF, store index and reload

In [24]:
documents1 = SimpleDirectoryReader( input_files=["newfile1.pdf"]).load_data()
index1 = VectorStoreIndex.from_documents(documents1)

# Specify the directory to persist the index
persist_dir = "./index_storage"

# Persist the index to the specified directory
index1.storage_context.persist(persist_dir=persist_dir)

In [27]:
from llama_index.core import load_index_from_storage
import os

# Check if the index storage directory exists
if os.path.exists(persist_dir):
    # Rebuild the storage context with the directory where the index was persisted
    storage_context = StorageContext.from_defaults(persist_dir=persist_dir)

    # Load the index from storage
    index2 = load_index_from_storage(storage_context)
else:
    print("Index storage not found. Please check the directory path.")

In [28]:
query1="what all information this document provide?"
query_engine1=index2.as_query_engine()
answer=query_engine1.query(query1)

# print(answer.get_formatted_sources())
print("query was:", query1)
print("answer is:", answer)


query was: what all information this document provide?
answer is: This document provides the text of the Hanuman Chalisa in English, along with a description in English. It describes the qualities and attributes of Hanuman, such as his strength, wisdom, and devotion to Lord Ram. It also mentions the benefits of reciting the Hanuman Chalisa, such as protection from evil forces and the fulfillment of desires.


###Query pdf with LLama index and pyMuPDF

In [ ]:
# !pip install llama-index docx2txt pypdf python-dotenv llama-index-embeddings-azure-openai llama-index-llms-azure-openai
!pip install fitz
!pip install frontend
# !pip install PyMuPDF

In [2]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from dotenv import dotenv_values

config=dotenv_values("val.env")

azure_openai_key = config["API_KEY"]
azure_openai_endpoint = config["ENDPOINT"]
azure_gpt_model_name=config["GPT_Model_Name"]
gpt_deployment_name=config["Deployment_Name"]
api_version=config['Model_Version']
azure_embed_model_name=config["Embed_Model_Name"]
embedding_deployment_name=config["Embed_Deployment"]

llm = AzureOpenAI(
    model=azure_gpt_model_name,
    deployment_name=gpt_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

embed_model = AzureOpenAIEmbedding(
    model=azure_embed_model_name,
    deployment_name=embedding_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [5]:
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.readers.file import PyMuPDFReader
# Load your document
loader = PyMuPDFReader()
documents = loader.load(file_path="/content/newfile.pdf")

# Create a vector store index
service_context = ServiceContext.from_defaults()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)


RuntimeError: Directory 'static/' does not exist